In [2]:
import numpy as np
import tensorflow as tf
from functools import reduce
import matplotlib.pyplot as plt
import random

gloabal_seed = round(random.random()*100000)

In [3]:
np.random.seed(gloabal_seed)
tf.set_random_seed(gloabal_seed)
sess = tf.InteractiveSession()
num_features = 3
num_samples = 5
state_size = 4
num_of_operations = 3
batch_size = num_samples
max_num_features = 10
datatype = tf.float64


#model placeholders
batchX_placeholder = tf.placeholder(datatype, [batch_size, None], name="batchX")
batchY_placeholder = tf.placeholder(datatype, [batch_size, None], name="batchY")

def add(vec):
    return reduce((lambda x, y: x + y),vec)

def samples_generator(fn, shape, rng, seed=None):
    '''
    Generate random samples for the model:
    @fn - function to be applied on the input features to get the ouput
    @shape - shape of the features matrix (num_samples, num_features)
    @rng - range of the input features to be generated within (a,b)
    @seed  - generation seed
    Outputs a tuple of input and output features matrix
    '''
    x = (rng[1] - rng[0]) * np.random.random_sample(shape) + rng[0]
    y = np.apply_along_axis(fn, 1, x).reshape((shape[0],-1))
    z = np.zeros((shape[0],shape[1] - y.shape[1]))
    y = np.concatenate((y, z), axis=1)
    '''
    j = -1
    for i,x_i in enumerate(x):
        j = j +1 
        if j > num_features -1: j = 0
        print(i,j)
        x[i,j] = 0
    '''

    return x,y


_x,_y = samples_generator(add, (num_samples, num_features) , (0, 100))

def tf_multiply(inpt):
    return tf.reshape( tf.reduce_prod(inpt, axis = 1), [batch_size, -1])
num_features
def tf_add(inpt):
    return  tf.reshape( tf.reduce_sum(inpt, axis = 1), [batch_size, -1])

def tf_stall(a):
    return a

W_te = tf.Variable(tf.truncated_normal([state_size, num_of_operations], -1*5, 5, dtype=datatype),dtype=datatype, name="W2")

x = tf.Variable(_x, dtype=datatype)
y = tf.Variable(_y, dtype=datatype)
W_te.initializer.run()
x.initializer.run()
y.initializer.run()

gloabal_seed

49723

In [ ]:
x.eval()

In [ ]:
def not_zero(inpt, pinpt=None):
        greater = tf.greater(inpt,tf.zeros_like(inpt, dtype=datatype))
        less = tf.less(inpt, tf.zeros_like(inpt, dtype=datatype))
        not_zero = tf.logical_or(greater, less)
        return not_zero


def tf_multiply(inpt, pinpt=None):
        result = tf.reduce_prod(inpt, axis = 1, name = "tf_mult")
        reshape = tf.reshape(result , [batch_size, -1], name = "tf_mult_reshape")
        pad_res = tf.pad(reshape, [[0,0],[0,num_features - 1]], "CONSTANT", name="tf_mult_pad")
        return pad_res

def tf_divide(inpt, pinpt=None):
        repriocal = tf.reciprocal(inpt)
        reg_slice = tf.slice(inpt, [0,0], [batch_size,1])
        repr_slice = tf.slice(repriocal, [0,1], [batch_size,num_features-1])
        intp  = tf.concat([reg_slice, repr_slice],1)
        masked_ones = tf.where(tf.is_inf(intp), tf.ones_like(inpt, dtype=datatype), intp)
        return tf_multiply(masked_ones)



equl = tf_divide(x)
equl.eval()

In [ ]:
add    = tf_add(x)
mult   = tf_multiply(x)
stall  = tf_stall(x)
values = tf.concat([add, mult, stall], 1)
#print(values.eval())
#padding = tf.constant()
print(add.eval())
print(mult.eval())
print(stall.eval())

In [ ]:
init_state = tf.Variable(np.zeros([batch_size, state_size]), dtype=datatype)
argmax_dum = tf.Variable([2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],  dtype=datatype)
W = tf.Variable(np.random.rand(state_size+num_features, state_size), dtype=datatype)
b = tf.Variable(np.zeros((state_size)), dtype=datatype)

W2 = tf.Variable(np.random.rand(state_size, num_of_operations),dtype=datatype)
b2 = tf.Variable(np.zeros((num_of_operations)), dtype=datatype)

dummy_matrix = tf.zeros([batch_size, num_features], datatype, name="dummy_constant")

init_state.initializer.run()
argmax_dum.initializer.run()
W.initializer.run()
b.initializer.run()
W2.initializer.run()
b2.initializer.run()
#dummy_matrix.initializer.run()

current_state = init_state
current_input = x
      
input_and_state_concatenated = tf.concat([current_input, current_state], 1)  # Increasing number of columns
next_state = tf.tanh(tf.matmul(input_and_state_concatenated, W) + b)  # Broadcasted addition
current_state = next_state
    
    #calculate softmax and produce the mask of operations
logits  = tf.matmul(next_state, W2) + b2 #Broadcasted addition
softmax = tf.nn.softmax(logits)
softmax_10000 = tf.nn.softmax(logits)
softplus = tf.nn.softplus(logits)
softsign = tf.nn.softsign(logits)
argmax  = tf.argmax(softmax, 1)

#lengths_transposed = tf.expand_dims(argmax, 0)
#onehot  = tf.one_hot(argmax, num_of_operations)
stall_width = tf.shape(stall)[1] - 1 
stall_select = tf.slice(softmax, [0,2], [batch_size,1])
multiples = [1, stall_width]
stall_mask = tf.tile(stall_select, multiples)
mask = tf.concat([softmax, stall_mask], 1)
#mask = tf.cast(mask, tf.bool)
#apply mask
#output = tf.boolean_mask(values,mask)
output = values * mask
nn_soft = tf.nn.softmax(logits)
output_width = tf.shape(output)[1]
#zeros_slice = tf.slice(dummy_matrix, [0,0], [batch_size, max_num_features - output_width])
#dum_plus_input = tf.concat([output, zeros_slice], 1)
#print(dum_plus_input.eval())

'''
print(x.eval())
print("stall_select")
print(stall_select.eval())
print("mask")
print(mask.eval())
print("logits")
print(logits.eval())
print("softmax")
print(softmax.eval())
print("softplus")
print(softplus.eval())
print("softsign")
print(softsign.eval())
print("softmax*10000")
print(softmax_10000.eval())
print("onehot")
print(onehot.eval())
'''
#print(mask.eval())
#print(mask.eval())
#print(output.eval())
print(softmax.eval())

In [ ]:
add_softmax   = tf.slice(softmax, [0,0], [batch_size,1])
mult_softmax  = tf.slice(softmax, [0,1], [batch_size,1])
stall_softmax = tf.slice(softmax, [0,2], [batch_size,1])

'''
print(add_softmax.eval())
print(mult_softmax.eval())
print(stall_softmax.eval())
'''
print(softmax.eval())
print(softmax[1].eval())
print(softmax[2].eval())

add_width   = tf.shape(add)[1]
mult_width  = tf.shape(mult)[1]
stall_width = tf.shape(stall)[1]

add_final = tf.pad(add, [[0,0],[0,num_features - add_width]], "CONSTANT") 
mult_final = tf.pad(mult, [[0,0],[0,num_features - mult_width]], "CONSTANT") 
stall_final = tf.pad(stall, [[0,0],[0,num_features - stall_width]], "CONSTANT")

#print(softmax.eval())

print(add_final.eval())
print(mult_final.eval())
print(stall_final.eval())

#tf.multiply()
add_final   = tf.multiply(add_final, softmax[0])
mult_final  = tf.multiply(mult_final, softmax[1])
stall_final = tf.multiply(stall_final, softmax[2])

print(add_final.eval())
print(mult_final.eval())
print(stall_final.eval())

In [ ]:
add_width   = tf.shape(add)[1]
mult_width  = tf.shape(mult)[1]
stall_width = tf.shape(stall)[1]


add_final   = add * add_softmax
mult_final  = mult * mult_softmax
stall_final = stall * stall_softmax

##conact add and mult results with zeros matrix
add_final = tf.concat([add_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - add_width])], 1) 
mult_final = tf.concat([mult_final, tf.slice(dummy_matrix, [0,0], [batch_size, num_features - mult_width])], 1) 

output_final = add_final + mult_final + stall_final 

print(add_width.eval())
print(mult_width.eval())
print(stall_width.eval())


print(add.eval())
print(mult.eval())
print(stall.eval())

print(add_softmax.eval())
print(mult_softmax.eval())
print(stall_softmax.eval())

print(add_final.eval())
print(mult_final.eval())
print(stall_final.eval())

print(output_final.eval())

In [ ]:

def custom_softmax(inpt):
    sum_exponents = tf.reduce_sum(tf.exp(inpt), axis = 1)
    soft_max = tf.log(sum_exponents)
    soft_max = tf.reshape( soft_max, [batch_size, -1])
    print(soft_max.eval())
    matrix_width = tf.cast(tf.shape(inpt)[1], tf.int32)
    multiples = [1, matrix_width]
    soft_max = tf.tile(soft_max, multiples)
    return  tf.reshape( inpt / soft_max, [batch_size, -1])

def custom_softmax2(inpt):
    sum_exponents = tf.reduce_sum(tf.exp(inpt), axis = 1)
    soft_max = sum_exponents
    soft_max = tf.reshape( soft_max, [batch_size, -1])
    print(soft_max.eval())
    matrix_width = tf.cast(tf.shape(inpt)[1], tf.int32)
    multiples = [1, matrix_width]
    soft_max = tf.tile(soft_max, multiples)
    return  tf.reshape( tf.exp(inpt) / soft_max, [batch_size, -1])
x_cust = custom_softmax(x)
x_cust2 = custom_softmax2(x)
print(x.eval())
#print(x_cust2.eval())
print(tf.nn.softmax(x*10000).eval())

In [ ]:
math_error = tf.multiply(tf.constant(0.5, dtype=datatype), tf.square(tf.subtract(tf.reduce_sum(output, axis = 1, name="red_output") , batchY_placeholder, name="sub_otput_batchY"), name="squar_error"), name="mult_with_0.5")

total_loss = tf.reduce_sum(math_error, name="red_total_loss")

grads = tf.gradients(total_loss, [W,b,W2,b2], name="comp_gradients")
train_step = tf.train.AdamOptimizer(0.1, name="AdamOpt").minimize(total_loss, name="min_loss")

In [ ]:
def calc_loss(output):
    #reduced_output = tf.reshape( tf.reduce_sum(output, axis = 1, name="red_output"), [batch_size, -1])
    math_error = tf.multiply(tf.constant(0.5, dtype=datatype), tf.square(tf.subtract(output , y, name="sub_otput_batchY"), name="squar_error"), name="mult_with_0.5")
    
                             
    total_loss = tf.reduce_sum(math_error, name="red_total_loss")
    return total_loss, math_error


loss, mathh = calc_loss(output_final)

In [ ]:
mathh.eval()

In [ ]:
prng = np.random.RandomState(1234)

In [ ]:
sess.close()

In [ ]:
prng.random_sample((10,5))

In [ ]:
_x[0:2]

In [ ]:
q = tf.FIFOQueue(capacity=2, dtypes=datatype)
q.enqueue(x)
q.dequeue().eval()

In [ ]:
for batch_idx in range(num_batches):
            start_idx = batch_size * batch_idx
            end_idx   = batch_size * batch_idx + batch_idx

            batchX = x[start_idx:end_idx]
            batchY = y[start_idx:end_idx]
            

In [ ]:
_x_ = tf.constant(_x, dtype=datatype, name="ROM")


In [ ]:
_x_.eval()

In [5]:
from ops import Operations
import pickle
from data_gen import samples_generator, split_train_test, np_avg_val
import tensorflow as tf
#path = '/home/user/neural_program_synthesis/models/summaries/RNN/np_mult-5ops/200_state_size-1500_num_samples-100_batch_size-0.005_learning_rate-1000.0_grad_norm-3_num_features-True_norm-True_share_state-3388_seed/'
#get the global configuration
#cfg_path = path+'cfg.p'
#cfg = pickle.load(open(cfg_path, 'rb'))

cfg = dict(
        num_samples = 100,
        batch_size  = 100,
        num_features = 4,
        datatype = tf.float64,
        train_fn = np_avg_val,
        samples_value_rng = (-100,100),
        seed = 12345,
        test_ratio = 0
    )

class Self:
    cfg = cfg

ops = Operations(cfg)
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
ops = Operations(cfg)
_sum = ops.ops[3](x).eval()
_len = ops.ops[0](x).eval()
_con = ops.ops[2](_sum,_len).eval()
_avg = ops.ops[1](_con).eval()
_avg

array([[-18.27646312,   0.        ,   0.        ,   0.        ],
       [ 39.04806743,   0.        ,   0.        ,   0.        ],
       [ 55.57490269,   0.        ,   0.        ,   0.        ],
       [-46.50262143,   0.        ,   0.        ,   0.        ],
       [ 68.51607053,   0.        ,   0.        ,   0.        ],
       [  7.65563167,   0.        ,   0.        ,   0.        ],
       [ 42.01109932,   0.        ,   0.        ,   0.        ],
       [-10.55218523,   0.        ,   0.        ,   0.        ],
       [ -2.67993828,   0.        ,   0.        ,   0.        ],
       [ 13.5839632 ,   0.        ,   0.        ,   0.        ],
       [ 11.23653494,   0.        ,   0.        ,   0.        ],
       [-29.74259691,   0.        ,   0.        ,   0.        ],
       [-20.66225504,   0.        ,   0.        ,   0.        ],
       [ 22.83308101,   0.        ,   0.        ,   0.        ],
       [ 31.33351491,   0.        ,   0.        ,   0.        ],
       [-27.22215938,   0

In [6]:
y - _avg

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  7.10542736e-15,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  0.00000000e

In [ ]:
def tf_inpt_len(self,inpt, mem_sel=None):
        with tf.name_scope("tf_inpt_len"):
            masked_zeros = tf.where( tf.equal(inpt,tf.zeros_like(inpt, dtype=self.cfg['datatype'])), tf.ones_like(inpt, dtype=self.cfg['datatype']), inpt, name="clean_zeros")
            inpt_ones = tf.divide(masked_zeros, masked_zeros, name="produce_ones")
            result = tf.reduce_sum(inpt_ones, axis = 1, name = "len_reduce")
            reshape = tf.reshape(result, [self.cfg['batch_size'], -1], name = "reshape")
            pad_res = tf.pad(reshape, [[0,0],[0,self.cfg['num_features'] - 1]], "CONSTANT", name="pad")
            return  pad_res

In [ ]:
x,y = samples_generator(np_avg_val, (100, 4) , (-100, 100), 100)
x_train, x_test, y_train, y_test = split_train_test (x, y , 0.33)
#x

In [ ]:


_x = tf.Variable(x, dtype=datatype)
_y = tf.Variable(y, dtype=datatype)
_x.initializer.run()
_y.initializer.run()

tf_inpt_len(Self,_y, _x).eval()


In [ ]:
from ops import Operations
ops.num_of_ops

In [ ]:
tf.ones_like(_x, dtype=cfg['datatype'],  name="tf_inpt_len_assign_ones").eval()

In [ ]:
_x.get_shape()